In [5]:
import pandas as pd
import re

def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', text)
    return result


df = pd.read_csv("review_data.csv")
df["ko_text"] = df["review"].apply(lambda x: text_cleaning(x))

df = df[df['ko_text'].str.len() > 0]
df.head()

,score,review,y,ko_text
1,4,녹는다 녹아,1,녹는다 녹아
2,4,좋아요,1,좋아요
3,5,흠잡을곳 없는 반찬이 정갈합니다,1,흠잡을곳 없는 반찬이 정갈합니다
4,5,코로나 땜에 걱정했는데 방역수칙도 잘 지키시고 살치실이랑 등심 부드러워서 너무 좋아...,1,코로나 땜에 걱정했는데 방역수칙도 잘 지키시고 살치실이랑 등심 부드러워서 너무 좋아...
5,5,살치살 미쳤네요.. 대박입니다 퀄리티 굳이에요~,1,살치살 미쳤네요 대박입니다 퀄리티 굳이에요


In [14]:
from konlpy.tag import Okt

# konlpy라이브러리로 텍스트 데이터에서 형태소를 추출합니다.
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word,tag) for word, tag in pos]
    return pos

result = get_pos(df['ko_text'].values[0])
print(result)

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.